In [18]:
#pip install tmdbsimple
import pandas as pd
import numpy as np
from numpy.random import shuffle
from sklearn import preprocessing
from sklearn.naive_bayes import MultinomialNB
import tmdbsimple as tmdb
tmdb.API_KEY = 'abcd9827ce3fb2298891370ff0cd0e58'

In [19]:
def takeInfos(id):
    atores = []
    crews = []
    keywords = []
    sTitles = []
    sPopularity = []
    sMedia = []

    movie = tmdb.Movies(id)

    response = movie.info()
    popularity = round(movie.popularity)
    media = round(movie.vote_average)
    title = movie.title
    for ator in movie.credits()['cast']: atores.append(ator['name'])
    for crew in movie.credits()['crew']: crews.append(crew['name'])
    for keyword in movie.keywords()['keywords']: keywords.append(keyword['name'])
    for similar in movie.similar_movies()['results']: 
        sTitles.append(similar['title'])
        sPopularity.append(round(similar["popularity"]))
        sMedia.append(round(similar["vote_average"]))
    
    return title,popularity,media,atores,crews,keywords,sTitles,sPopularity,sMedia

def NormalizeCat(X): #Não esta sendo usado
    global perda
    i = 0
    n = len(X)
    limit = 300
    if len(X)>limit:perda+=1
    while(len(X)<limit):
        if i == n: i = 0
        X.append(X[i])
        i+=1
    while(len(X)>limit):
        X.pop()
    return X
        

In [20]:
class Encoder:
    def __init__(self,lista):
        self.Codex = []
        self.atualIndex = 1
        self.fit(lista)
        
    def fit(self,lista):
        for item in lista:
            onList = False
            for code in self.Codex:
                if item == code[0]: onList = True
            if not onList:
                self.Codex.append([item,self.atualIndex])
                self.atualIndex+=1
    
    def transform(self,lista):
        resposta = []
        for item in lista:
            for code in self.Codex:
                if item == code[0]: resposta.append(code[1])
        return resposta
    
    def fit_transform(self,lista):
        self.fit(lista)
        return self.transform(lista)
    
    def inverse_transform(self,lista):
        resposta = []
        for item in lista:
            for code in self.Codex:
                if item == code[1]: resposta.append(code[0])
        if len(lista)==1: return resposta[0]
        return resposta
    
    def show(self):
        print(self.Codex)
    
    def Feed(self,id):
        title,popularity,media,atores,crews,keywords,sTitles,sPopularity,sMedia = takeInfos(id)
        self.fit([title])
        self.fit(sTitles)
        self.fit(atores)
        self.fit(crews)
        self.fit(keywords)

        Atores = self.transform(atores)
        Crews = self.transform(crews)
        Keywords = self.transform(keywords)
        X = self.fit_transform(["P"+str(popularity)])
        if len(Crews)>0:X.append(Crews[0])
        else: X.append(X[-1])
        for i in range(3):
            if len(Atores)>(i): X.append(Atores[i])
            else: X.append(X[-1])
            if len(Keywords)>(i): X.append(Keywords[i])
            else: X.append(X[-1])
        y = [media]
        X = [X]
        return X,y

In [76]:
IDlist = list(movies["id"])
n = 3000
IDTrainList = IDlist[:n]
IDTestList = IDlist[n:]

In [21]:
credits = pd.read_csv("tmdb_5000_credits.csv")
movies = pd.read_csv("tmdb_5000_movies.csv")

In [22]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [23]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [24]:
Naive = MultinomialNB()

In [ ]:
#NÃO RODAR ISSO SE NÃO FOR MUUUITO NECESSARIO
Codex = Encoder(['M0',"M1","M2",'M3','M4','M5','M6','M7','M8','M9','M10'])
perda = 0
X,y = Codex.Feed(603)
Naive.fit(X,y)
#else: perda+=1

C:\Users\Bruno\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:455: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [ ]:
n = 300
i = 0
shuffle(IDTrainList)
for idd in IDlistTrainList:
    X,y = Codex.Feed(idd)
    Naive.partial_fit(X,y)
    i += 1
    print(str(i)+"/"+str(n))
    if i == n:break

C:\Users\Bruno\Anaconda3\lib\site-packages\sklearn\naive_bayes.py:455: RuntimeWarning: divide by zero encountered in log
  self.class_log_prior_ = (np.log(self.class_count_) -


1/300
2/300
3/300
4/300
5/300
6/300
7/300
8/300
9/300
10/300
11/300
12/300
13/300
14/300
15/300
16/300
17/300
18/300
19/300
20/300
21/300
22/300
23/300
24/300
25/300
26/300
27/300
28/300
29/300
30/300
31/300
32/300
33/300
34/300
35/300
36/300
37/300
38/300


In [ ]:
i = 0
n = 500
s8 = 0
n8 = 0
e1 = 0
em1 = 0
a = 0

shuffle(IDTestList)
for idd in IDTestList: 
    X,y = Codex.Feed(idd)
    saida = Naive.predict(X)
    
    if saida != 8: n8 += 1
    else: s8 += 1
    
    if saida == y: a += 1
    elif (saida == y[0]+1 or saida == y[0]-1): e1 += 1
    else: em1 +=1

    i += 1
    print(str(i)+"/"+str(n))
    if i == n:break
        
print("\n a: {0}".format(a))
print("e1: {0}".format(e1))
print("em1: {0}".format(em1))
print("\n n8: {0}".format(n8))
print(" s8: {0}".format(s8))

In [ ]:
search = tmdb.Search()
response = search.movie(query='The Man Who Invented')
for s in search.results: print(s['title'], s['id'], s['release_date'], s['popularity'], round(s['vote_average']))